# Семинар 8: решающие деревья

## Теоретическая часть

### Предсказания в решающих деревьях
__Вспомнить из лекции:__
* Что такое решающее дерево? 
* Что такое предикат? Каки бывают предикаты? Какие предикаты чаще всего используются? 
* В чем отличие внутренних и листовых вершин решающего дерева?
* Как выполнить предсказание с помощью решающего дерева в задачах многоклассовой классификации и регрессии?

__На что обратить внимание__:
* В линейной классификации мы рассматривали бинарную и многоклассовую классификацию отдельно, потому что предсказание и обучение выполняется по-разному для этих двух случаев. В решающих деревьях бинарная и многоклассовая классификация выполняются одинаково.
* Предсказания можно сделать для любой точки признакового пространства, то есть для любого возможного объекта, а не только для объекта обучающей выборки. 
* Признаки в предикатах различных вершин могут повторяться.

#### Задача 1
Рассмотрим задачу классификации на три класса по двум признакам и следующее решающее дерево: 

<div>
<img src="tree_class.png" width="200"/>
</div>

Какое предсказание это решающее дерево вернет для объекта $x=(7, 1.5)$? Под d1 и d2 подразумеваются первый и второй признак.


#### Задача 2. 
Нарисуйте разделяющую поверхность для решающего дерева из предыдущей задачи.

_На что обратить внимание_: в этой задаче у нас нет обучающей выборки, потому что мы предполагаем, что решающее дерево уже задано, и наша задача - как бы сделать предсказания для всех возможных объектов пространства (отсюда и возникает разделяющая поверхность). Об обучении решающего дерева по конкретной выборке мы поговорим в следующем разделе.

#### Задача 3
Рассмотрим задачу регрессии по одному признаку. Визуализируйте решающее правило    для следующего решающего дерева:

<div>
<img src="tree_reg.png" width="200"/>
</div>

#### Задача 4
Приведите пример решающего дерева, которое даст нулевую ошибку в задаче классификации по двум признакам, изображенной ниже. Изобразите само решающее дерево, а также изобразите получающуюся разделяющую поверхность на рисунке. Используйте предикаты вида [j-й признак < t].

<div>
<img src="ideal_task.png" width="200"/>
</div>


### Обучение решающего дерева
__Непрерывная и дискретная оптимизация.__ Для обучения моделей, как правило, используются методы оптимизации. В зависимости от того, по каким переменным нужно выполнять оптимизацию, выделяют два вида оптимизации: непрерывную и дискретную. Непрерывная оптимизация выполняется по вещественным числам, часто здесь используют градиентные методы. С помощью непрерывной оптимизации мы обучали линейные модели. Дискретная оптимизация выполняется по конечным множествам, для этого необходимо делать перебор по элементам множества. Делать полный перебор может быть очень долго, поэтому придумывают более эффективные алгоритмы, например один из самых простых - жадный алгоритм. С помощью жадной дискретной оптимизации мы будем обучать решающие деревья. В решающих деревьях нужно выбрать одно из всех возможных разбиений признакового пространства на области. Несмотря на то, что всего разбиений бесконечное число, разбиений, приводящих к разным предсказаниям на обучающей выборке, конечное число - между ними и нужно выбрать одно.

__Вспомнить из лекции:__
* Рекурсивный алгоритм построения решающего дерева.
* Какие вы знаете критерии останова при построении решающего дерева (когда выборку не нужно делить, а нужно сделать лист)?
* Как выбрать, какое предсказание делать в листе для задачи регрессии и задачи классификации?

__Критерии информативности: интуиция.__ 
Когда мы строим одну вершину решающего дерева, мы разбиваем выборку на две подвыборки: одна подвыборка будет использоваться при построении левого поддерева, вторая - при построении правого поддерева. 
Какую цель мы преследуем, когда разбиваем выборку на две подвыборки? Каких свойств мы хотим от получающихся подвыборок? Можно придумать разные ответы, но на практике хорошо работает следующее желаемое свойство: мы хотим, чтобы ответы в обеих подвыборках были как можно менее вариативны. В идеальной ситуации в каждой подвыборке у всех объектов одинаковые ответы, и дальше строить дерево не нужно (каждая подвыборка станет листом). На практике так обычно не получается, иначе это бы уже не было задачей машинного обучения. Однако мы можем постепенно уменьшать вариативность ответов в подвыборках. Итак, зафиксируем следующую постановку задачи: у нас есть вектор правильных ответов $Y_v$ (ответы на всех объектах, попавших в вершину $v$), и мы хотим измерить вариативность этих ответов.

Как измерить вариативность в задаче регрессии? Для этого можно использовать среднеквадратичное отклонение:
$$H(Y_v) = \sum_{i=1}^{|Y_v|} (Y_{v, i} - \bar Y_{v})^2,$$
где $\bar Y_{v}$ - среднее вектора $Y_{v}$.
Можно выбирать аналогичные меры разброса в зависимости от функции потерь, используемой в задаче. Например, при использовании MAE можно заменить квадрат на модуль, а среднее - на медиану.

Как измерить вариативность в задаче классификации? Здесь обычно подходят следующим образом: на основе вектора $Y_v$, состоящего из меток классов $1 \dots K$, вычисляют доли каждого класса: $(p_1, \dots, p_K)$. Если все элементы вектора одинаковые (вариативность наименьшая), то среди $p_k$ будет одна 1, оостальные значения 0 (назовем это случай а). При наименьшей вариативности вектора $Y_v$ все $p_k \approx \frac 1 k$ (случай б). Осталось придумать критерий, зависящий от $(p_1, \dots, p_K)$, который минимален в случае а и максимален в случае б. Таким критерием является, например, энтропийный критерий:
$$H(Y_v) = H(p_1, \dots, p_K) = - \sum_{k=1}^K p_k \log p_k$$
или критерий Джини:
$$H(Y_v) = H(p_1, \dots, p_K) = \sum_{k=1}^K p_k (1-p_k).$$

В итоге, мы можем измерить вариативность подвыборок, из которых будет строиться левое и правое поддерево. Чтобы построить вершину $v$, мы будем перебирать все возможные признаки $j$ и все возможные пороги $t$ (всего $\ell d$ вариантов, $\ell$ - число объектов, $d$ - число признаков), для каждой пары $(j, t)$ мы получим разбиение выборки на две части, им соответствуют векторы правильных ответов $Y_\ell$ и $Y_r$. Далее нам нужно сравнить все разбиения и выбрать лучшее. Сравнивать разбиения удобно, когда есть один критерий;  у нас же пока два критерия. Скомбинируем вариативность обеих выборок в одном критерие:
$$Q(Y_v, j, t) = \frac {|Y_\ell|}{|Y_v|} H(Y_\ell) 
+ \frac {|Y_r|}{|Y_v|} H(Y_r) \rightarrow \min_{j, \,t}$$

Веса $\frac {|Y_\ell|}{|Y_v|}$ и $\frac {|Y_r|}{|Y_v|}$ вводятся для того, чтобы не поощрять отделение одного объекта в отдельную вершину: для таких подвыборок вариативность вектора правильных ответов низкая, и без перевзвешивания алгоритм всегда выбирал бы подобные разбиения. А это нежелательно, потому что вершины из одного объекта позволяют запоминать ответы, что ведет к переобучению. 

Итоговый алгоритм построения вершины: перебрать все варианты $(j, t)$, для каждого посчитать критерий $Q(Y_v, j, t)$, выбрать пару с наибольшим значением критерия.

#### Задача 5
Предположим, мы решаем задачу классификации на три класса по четырем признакам и строим решающее дерево. При построении вершины $v$ мы имеем выборку из семи объектов:

| 1-й признак | 2-й признак | 3-й признак | 4-й признак | класс |
|-------------|-------------|-------------|-------------|-------|
| 5           | 8           | 8           | 2.5         | 2     |
| 3           | 3           | 7           | 7.7         | 1     |
| 6           | 7.7         | 1           | 1.1         | 3     |
| 3.3         | 1           | 2           | 1.2         | 1     |
| 24          | 3.9         | 5           | 3.9         | 1     |
| 12          | 10          | 10.1        | 8           | 2     |
| 1           | 2           | 2           | 9.1         | 1     |

Какой из предикатов лучше: [1-й признак < 6] или [3-й признак < 5] по критерию Джини?


#### Вопрос: Что является параметрами и гиперпараметрами решающих деревьев?

__Решение:__

Когда мы говорим о семействе алгоритмов машинного обучения, мы задаем алгоритм, как по входному объекту сделать предсказание. В этом алгоритме обычно участвуют две группы величин: признаки объекта (даны) и параметры (неизвестные величины). В ходе обучения мы настраиваем параметры по обучающей выборке, чтобы получить конкретный алгоритм предсказания. В алгоритмах предсказания и обучения могут также присутствовать величины, которые мы должны задать сами, не по данным; их называют гиперпараметры.

В решающих деревьях мы задали алгоритм предсказания как проход вниз по дереву, в каждой вершине которого находится предикат или предсказание. Вот эти придикаты и предсказания и есть параметры решающего дерева (они настраиваются по обучающей выборке). Для самого популярного вида предикатов [j-й признак < t] параметрами являются признаки и пороги в каждой внутренней вершине, а также константные предсказания в листовых вершинах.

Гиперпараметрами решающего дерева являются все детали обучения: критерии останова, критерий информативности и т. д. Используя разные критерии останова и криерии информативности, мы получим разные решающие деревья. Какие лучше, нужно выбирать по валидационной выборке или кросс-валидации.




Рассмотрим модельную задачу регрессии. Объектами будут являться точки на плоскости (т.е. каждый объект описывается 2 признаками), целевая переменная — расстояние от объекта до точки (0, 0).

Напишем вспомогательную функцию, которая будет возвращать решетку для дальнейшей красивой визуализации

In [ ]:
def get_grid(data):
    x_min, x_max = data[:, 0].min() - 1, data[:, 0].max() + 1
    y_min, y_max = data[:, 1].min() - 1, data[:, 1].max() + 1
    return np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))

Сгенерируем выборку

In [ ]:
data_x = np.random.normal(size=(100, 2))
data_y = (data_x[:, 0] ** 2 + data_x[:, 1] ** 2) ** 0.5
plt.figure(figsize=(8, 8))
plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=100, cmap='spring')

Обучим дерево на сгенерированных данных и предскажем ответы для каждой точки решетки

In [ ]:
from sklearn.tree import DecisionTreeRegressor

clf = DecisionTreeRegressor()
clf.fit(data_x, data_y)

xx, yy = get_grid(data_x)
print(np.c_[xx.ravel(), yy.ravel()])

predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

plt.figure(figsize=(8, 8))
plt.pcolormesh(xx, yy, predicted, cmap='spring')
plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=100, cmap='spring')

### Задание

Сейчас мы сгенерировали 100 точек из нормального распределения и обучили решающее дерево на них. Сгенерируйте 300 точек из нормального распределения, обучите на них дерево и выведите на экран результат (как на картинке выше).

Сгенерированные точки и расстояние до точек сохраните в массивы data_x300, data_y300, для обучения и предсказания используйте эти массивы.

Улучшилось ли предсказание алгоритма на решётке? (т.е. стала ли раскраска всей плоскости более правильной?)

In [ ]:
#your code here

Вернёмся к исходным данным (100 точек).

Посмотрим как будут выглядеть разделяющая поверхность в зависимости от 
- минимального количества объектов в листе
- максимальной глубины дерева

In [ ]:
plt.figure(figsize=(18, 18))
for i, max_depth in enumerate([1, 2, 4, 6]):
    for j, min_samples_leaf in enumerate([1, 5, 10, 15]):
        clf = DecisionTreeRegressor(max_depth=max_depth, min_samples_leaf=min_samples_leaf)
        clf.fit(data_x, data_y)
        xx, yy = get_grid(data_x)
        predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
        
        plt.subplot2grid((4, 4), (i, j))
        plt.pcolormesh(xx, yy, predicted, cmap='spring')
        plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=30, cmap='spring')
        plt.title('max_depth=' + str(max_depth) + ', min_samples_leaf: ' + str(min_samples_leaf))

- Как влияет увеличение максимальной глубины и/или уменьшение минимального количества объектов выборки в листе на качество на обучающей выборке? на переобучение?

## Неустойчивость решающих деревьев

Решающие деревья — это алгоритмы, неустойчивые к изменениям обучающей выборки, т.е. при малейших её изменениях итоговый классификатор может радикально измениться.
Посмотрим, как будет меняться структура дерева при обучении на разных 90%-х подвыборках.

In [ ]:
plt.figure(figsize=(20, 6))
for i in range(3):
    clf = DecisionTreeRegressor(random_state=42)

    indices = np.random.randint(data_x.shape[0], size=int(data_x.shape[0] * 0.9))
    clf.fit(data_x[indices], data_y[indices])
    xx, yy = get_grid(data_x)
    predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    plt.subplot2grid((1, 3), (0, i))
    plt.pcolormesh(xx, yy, predicted, cmap='winter')
    plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=30, cmap='winter')

## Подбор параметров

Посмотрим на качество дерева в зависимости от параметров на одном из стандартных наборов данных - Бостонском датасете.

In [ ]:
from sklearn.datasets import load_boston

data = load_boston()
print(data.DESCR)

In [ ]:
X = data.data
y = data.target

X.shape

- будем оценивать качество алгоритма по кросс-валидации

Можем зафиксировать разбиение на фолды, чтобы затем каждый раз использовать одно и то же разбиение при кросс-валидации, это полезно при сравнении алгоритмов

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
cv = KFold(X.shape[0], shuffle=True, random_state=241)

Выведите качество DecisionTreeRegressor, обученного на данных X, y по кросс-валидации. В функции cross_val_score в качестве cv поставьте cv=cv, в качестве метрики - 'neq_mean_squared_error'

In [ ]:
#your code here

### Задание

Метрика MSЕ имеет не ограничена сверху. Поэтому для оценки качества алгоритма можно также пользоваться метрикой R2 (коэффициент детерминации), так как он не превышает 1 (и чем ближе к 1, тем лучше).

Выведите на экран значение R2 алгоритма ('r2').

In [ ]:
#your code here

Для сравнения качества модели при различных наборах параметров или для сравнения моделей на одном датасете можно использовать, как и раньше, MSE.

Будем подбирать параметры решающего дерева по сетке с целью увеличить качество алгоритма. Будем подбирать значения max_features и max_depth.

In [ ]:
from sklearn.metrics import SCORERS
SCORERS.keys()

Подберите по кросс-валидации оптимальные значения max_features и max_depth. В функции GridSearchCV в качестве cv поставьте заранее фиксированное разбиение (cv=cv), метрику качества используйте scoring='neq_mean_squared_error'

In [ ]:
params={'max_features': [None, 'log2', 'sqrt'], 
        'max_depth': [2, 4, 6, 8, 10, 20, 50]},

gs = GridSearchCV(DecisionTreeRegressor(), params, cv=3, n_jobs=-1)

gs.fit(X, y)

Выведем на экран средние значения и стандартные отклонения, полученные при GridSearch.

In [ ]:
means = gs.cv_results_['mean_test_score']
stds = gs.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, gs.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
            % (mean, std * 2, params))

# Задание

Теперь попробуем одновременно подбирать значения max_features, max_depth и min_samples_leaf. Ищите min_samples_leaf в диапазоне от 1 до 20 с шагом 1.

In [ ]:
#your code here

In [ ]:
gs.best_score_

Как в данной задаче зависит качество алгоритма от количества параметров, которые мы оптимизируем?